In [11]:
import psycopg2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
host = "dbinstance.c6phnxzyppjs.us-west-2.rds.amazonaws.com"
dbname = "muimui"
username = "marwa"
password = "muimuidb"
port = "5432"

sql_conn_string = "host='{}' dbname='{}' user='{}' password='{}' port='{}'".format(host, dbname, username, password, port)

In [3]:
conn = psycopg2.connect(sql_conn_string)
cur = conn.cursor()
rows = []
cmds = [f"""
        select * from products
        ;
        """]
out_collection = {}

for c in cmds:
    try:
        cur.execute(c)
        data = cur.fetchall()
        for row in data:
            rows.append(list(row))
        conn.commit()
    except psycopg2.ProgrammingError:
        print( """CAUTION FAILED: '%s' """ % c)
        conn.rollback()

In [4]:
names = ['ID','page_link','image_link', 'name', 'price', 'description' , 'brand']     
df = pd.DataFrame(rows , columns= names)
df.head()

,ID,page_link,image_link,name,price,description,brand
0,0,https://www.macys.com//shop/product/jm-collect...,https://slimages.macysassets.com/is/image/MCY/...,"Printed Scoop-Neck Top, Created for Macy's",34.5,Complete your casual wardrobe with this wear-e...,JM Collection
1,1,https://www.macys.com//shop/product/jm-collect...,https://slimages.macysassets.com/is/image/MCY/...,"Printed Scoop-Neck Top, Created for Macy's",34.5,Complete your casual wardrobe with this wear-e...,JM Collection
2,2,https://www.macys.com//shop/product/jm-collect...,https://slimages.macysassets.com/is/image/MCY/...,"3/4-Sleeve Novelty Printed Jacquard Top, Creat...",44.5,Slip into this essential JM Collection top any...,JM Collection
3,3,https://www.macys.com//shop/product/jm-collect...,https://slimages.macysassets.com/is/image/MCY/...,"3/4-Sleeve Novelty Printed Jacquard Top, Creat...",44.5,Slip into this essential JM Collection top any...,JM Collection
4,4,https://www.macys.com//shop/product/jm-collect...,https://slimages.macysassets.com/is/image/MCY/...,"Printed T-Shirt, Created for Macy's",34.5,A fashionable print lends an upgraded look and...,JM Collection


remove : string

In [5]:
dirty_words = ['bandeau','bikini','bodysuit','boyshort','Bra','breast','brief','Bustier','butt','camisole','charmel','chemise','cover-up','cups','fishnet','Garter','Girdles','gstring','Hipster','hosiery','intimates','jockey','Knickers','lingerie','loungewear','minimizer','minimizers','Nightgown','nude','nursing','One-Piece','pajama','Panty','pantyhose','plunge','pushup','robes','sensual','shapewear','sheer','sleepwear','slip','socks','swim','swimdress','swimsuit','swimwear','tankini','Thong','tights','tummy','undergarment','underwear','underwire','vagina','wireless','wonderbra']
dirty_words = [d.lower() for d in dirty_words]

In [10]:
good_words = ['top', 'shirt', 'skirt', 'pants', 'dress', 'blazer', 'coat', 'jacket','blouse', 'jeans', 'capri', 'jumpsuit','sweater', 'cardigan', 'tunic', 'hoodie','vest', 'trousers']
good_words = [d.lower() for d in good_words]

In [24]:
df['tot_desc'] = df['name'] + df['description'] + df['brand']
df['tot_desc'] = df['tot_desc'].astype(str)
df['tot_desc'] = df['tot_desc'].apply(lambda x: x.lower())

In [25]:
len(df)

30370

In [26]:
df['is_good'] = 0
df['is_bad'] = 0

In [27]:
count = 0
for i in range(len(df)):
    for d in good_words :
        if d in df.tot_desc[i]:
            df['is_good'][i] =1
            break 

In [28]:
df.is_good.sum()

27041

In [29]:
count = 0
for i in range(len(df)):
    for d in dirty_words :
        if d in df.tot_desc[i]:
            df['is_bad'][i] =1
            break 

In [33]:
df.is_bad.sum()

12123

In [34]:
df['is_nsfw'] = 0

In [44]:
df['is_nsfw'] = list(np.array((df.is_good == 0) & (df.is_bad == 1), dtype = int))

In [46]:
df.to_csv('products_safe.csv')

In [48]:
df.columns

Index(['ID', 'page_link', 'image_link', 'name', 'price', 'description',
       'brand', 'tot_desc', 'is_good', 'is_bad', 'is_nsfw'],
      dtype='object')

In [49]:
df = df[['ID', 'page_link', 'image_link', 'name', 'price', 'description',
       'brand','is_nsfw']]

In [50]:
df.to_csv('products_no_nsfw.csv')

In [36]:
temp = df[(df.is_good == 0) & (df.is_bad == 1)]
for i in range(len(temp)):
    print(temp.iloc[i,:].page_link )

https://www.macys.com//shop/product/i.n.c.-seamless-camisole-created-for-macys?ID=7914948&CategoryID=188851
https://www.macys.com//shop/product/i.n.c.-seamless-camisole-created-for-macys?ID=7914948&CategoryID=188851
https://www.macys.com//shop/product/tommy-hilfiger-logo-strap-overalls?ID=8056963&CategoryID=188851
https://www.macys.com//shop/product/tommy-hilfiger-logo-strap-overalls?ID=8056963&CategoryID=188851
https://www.macys.com//shop/product/lucky-brand-cotton-kimono?ID=8491126&CategoryID=188851
https://www.macys.com//shop/product/lucky-brand-cotton-kimono?ID=8491126&CategoryID=188851
https://www.macys.com//shop/product/lucky-brand-the-bermuda-denim-shorts?ID=8037571&CategoryID=188851
https://www.macys.com//shop/product/lucky-brand-the-bermuda-denim-shorts?ID=8037571&CategoryID=188851
https://www.macys.com//shop/product/style-co-patchwork-print-kimono-created-for-macys?ID=7536295&CategoryID=188851
https://www.macys.com//shop/product/style-co-patchwork-print-kimono-created-for-mac

https://www.macys.com//shop/product/wire-free-padded-bra?ID=7278052&CategoryID=188851
https://www.macys.com//shop/product/criss-cross-lace-panty?ID=7979317&CategoryID=188851
https://www.macys.com//shop/product/alfani-brushed-soft-knit-sleep-shorts-created-for-macys?ID=6600177&CategoryID=188851
https://www.macys.com//shop/product/hanky-panky-womens-panne-velvet-eyelash-lace-diamond-thong-3y1946?ID=7154054&CategoryID=188851
https://www.macys.com//shop/product/vanity-fair-breathable-luxe-seamless-comfort-wirefree-bra-72219?ID=6801484&CategoryID=188851
https://www.macys.com//shop/product/eileen-west-plus-size-lace-trimmed-printed-knit-nightgown?ID=7007790&CategoryID=188851
https://www.macys.com//shop/product/tommy-hilfiger-womens-seamless-logo-hipster-r17t042?ID=7214087&CategoryID=188851
https://www.macys.com//shop/product/contour-underwire-bra?ID=7278049&CategoryID=188851
https://www.macys.com//shop/product/skarlett-blue-womens-heartbreaker-lace-bikini-373162?ID=6789209&CategoryID=188851
